In [1]:
from kaggle_environments import evaluate, make, utils

env = make("connectx", debug=True)
env.render()

+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+



In [2]:
help(env)

Help on Environment in module kaggle_environments.core object:

class Environment(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, specification={}, configuration={}, steps=[], agents={}, interpreter=None, renderer=None, html_renderer=None, debug=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  clone(self)
 |      Returns:
 |          Environment: A copy of the current environment.
 |  
 |  play(self, agents=[], **kwargs)
 |      Renders a visual representation of the environment and allows interactive action selection.
 |      
 |      Args:
 |          **kwargs (dict): Args directly passed into render().  Mode is fixed to ipython.
 |      
 |      Returns:
 |          None: prints directly to an IPython notebook
 |  
 |  render(self, **kwargs)
 |      Renders a visual representation of the current state of the environment.
 |      
 |      Args:
 |          mode (str): html, ipython, ansi, human (default)
 |          **kwargs (d

In [3]:
import env_configurations
import games_configurations
env_name = 'connectx'
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)

print(obs_space)
print(action_space)

/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-

Box(7, 6, 3)
Discrete(7)


In [4]:
trainer = env.train([None, "random"])

In [5]:
import dqnagent
import yaml
from runner import Runner
from kaggle_env import KaggleEnv
import tensorflow as tf
import numpy as np

gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.2)



sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.2)))
with open('configs/connectx_ppo.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
    runner = Runner()
    runner.load(config)
    runner.sess = sess
    
player = runner.create_player()
#player.restore('nn/best_connectx')
player.restore('nn/connectx_ppoconnectx')
is_determenistic = True
def ppo_agent(observation, configuration):
    #self.agent.get_masked_action_values([obs],)
    mask = np.array(observation.board[0:7]) == 0
    obs = KaggleEnv.build_obs(True, True, observation)
    action = player.get_masked_action([obs], [mask], is_determenistic)
    if np.sum(observation['board']) == 0:
        return np.random.randint(7)
    if observation.board[action] != 0:
        print('wrong step')
        print(mask)
        print(action)
        for i in range(7):
            if observation.board[i] == 0:
                return i
    else:
        return int(action)
    
def ppo_agent2(observation, configuration):
    mask = np.array(observation.board[0:7]) == 0
    action = player.get_masked_action([KaggleEnv.build_obs(False, True,observation)], [mask], is_determenistic)
    if observation.board[action] != 0:
        print('wrong step')
        print(mask)
        for i in range(7):
            if observation.board[i] == 0:
                return i
    else:
        return int(action)


using masks for action
Instructions for updating:
Use `tf.cast` instead.

conv_name: conv2d
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.




Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from nn/connectx_ppoconnectx


In [6]:
#env.reset()
# Play as the first agent against default "random" agent.
env.run([ppo_agent, "negamax"])
#env.run([ "negamax", ppo_agent2])
env.render(mode="ipython", width=500, height=450)

In [7]:
def mean_reward(rewards):
    return sum(r[0] for r in rewards) / sum(r[0] + r[1] for r in rewards)
def mean_reward2(rewards):
    return 1.0 - sum(r[0] for r in rewards) / sum(r[0] + r[1] for r in rewards)


print("My Agent vs agent2:", mean_reward(evaluate("connectx", [ppo_agent, ppo_agent2], num_episodes=100)))

print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [ppo_agent, "random"], num_episodes=100)))
#print("My Agent vs Random Agent:", mean_reward2(evaluate("connectx", ["random", ppo_agent], num_episodes=100)))
#print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [ppo_agent2, "random"], num_episodes=100)))
print("My Agent vs Random Agent:", mean_reward2(evaluate("connectx", ["random", ppo_agent2], num_episodes=100)))
#print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [my_agent, my_agent], num_episodes=1000)))
#print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [my_agent2, my_agent2], num_episodes=1000)))
#print("My Agent vs Negamax Agent:", mean_reward2(evaluate("connectx", [ "negamax", ppo_agent], num_episodes=100)))
print("My Agent vs Negamax Agent:", mean_reward2(evaluate("connectx", [ "negamax", ppo_agent2], num_episodes=100)))
print("My Agent vs Negamax Agent:", mean_reward(evaluate("connectx", [ppo_agent, "negamax", ], num_episodes=100)))
#print("My Agent vs Negamax Agent:", mean_reward(evaluate("connectx", [ppo_agent2, "negamax", ], num_episodes=100)))





My Agent vs agent2: 0.53
My Agent vs Random Agent: 0.96
My Agent vs Random Agent: 0.88
My Agent vs Negamax Agent: 0.31000000000000005
My Agent vs Negamax Agent: 0.305


In [8]:
import tensorflow_probability as tfp
tfd = tfp.distributions
def test():
    dist = tfd.Multinomial(total_count=1., probs=[.5, .4, .1, 0.0])

    with tf.Session() as sess:

        g = dist.sample(10)
        results = sess.run([g])

    for r in results:
        print(r)

In [9]:
test()

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
